In [4]:
# glove embeddding
glove_twitter = 'http://nlp.stanford.edu/data/glove.twitter.27B.zip'
glove_crowl = 'http://nlp.stanford.edu/data/glove.840B.300d.zip'
#w2v_en = 'https://doc-14-3g-docs.googleusercontent.com/docs/securesc/4cvgiao1nb4te9ua4i7vv69oumat5dfg/t5p4t88g0t2rg8383c5aerjv2uvlg06h/1520100000000/06848720943842814915/16463238193964568227/0B7XkCwpI5KDYNlNUTTlSS21pQmM?e=download&nonce=cv5jml82qplcm&user=16463238193964568227&hash=g5prh0perrt6lf98c1unarl0q754iugf'
ft_fr = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.fr.zip'
ft_en = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.en.zip'
ft_gr = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.de.zip'
dep = 'http://u.cs.biu.ac.il/~yogo/data/syntemb/deps.words.bz2'

In [5]:
urls = [ft_en, glove_twitter, glove_crowl, ft_fr,ft_gr,dep]

In [6]:
for emb in urls:
    ! cd vocs/
    ! wget $emb

/bin/sh: line 0: cd: vocs/: No such file or directory
--2018-03-06 19:17:29--  https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.en.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.20.13
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.20.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10356881291 (9.6G) [application/zip]
Saving to: ‘wiki.en.zip’

wiki.en.zip         100%[===================>]   9.65G  18.2MB/s    in 8m 15s  

2018-03-06 19:25:45 (20.0 MB/s) - ‘wiki.en.zip’ saved [10356881291/10356881291]

/bin/sh: line 0: cd: vocs/: No such file or directory
--2018-03-06 19:25:45--  http://nlp.stanford.edu/data/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.twitter.27B.zip [follo

In [17]:
! unzip glove.840B.300d.zip

Archive:  glove.840B.300d.zip
  inflating: glove.840B.300d.txt     


In [15]:
! unzip wiki.de.zip

Archive:  wiki.de.zip
  inflating: wiki.de.vec             
  inflating: wiki.de.bin             


In [16]:
! unzip wiki.fr.zip

Archive:  wiki.fr.zip
  inflating: wiki.fr.vec             
  inflating: wiki.fr.bin             


In [20]:
! gzip -d GoogleNews-vectors-negative300.bin.gz

In [21]:
! bzip2 -d deps.words.bz2

In [22]:
import os
os.listdir()

['.ipynb_checkpoints',
 'wiki.en.vec',
 'wiki.de.bin',
 'deps.words',
 'words_glove_twitter_for_check_spell.pkl',
 'glove.twitter.27B.50d.txt',
 'wiki.en.bin',
 'glove.twitter.27B.100d.txt',
 'glove.840B.300d.txt',
 'wiki.de.vec',
 'wiki.fr.bin',
 'wiki.fr.vec',
 'glove.840B.300d.zip',
 'load_embs.ipynb',
 'words_for_check_spell.pkl',
 'glove.twitter.27B.200d.txt',
 'GoogleNews-vectors-negative300.bin',
 'glove.twitter.27B.25d.txt',
 'glove.twitter.27B.200d_w2v.txt']

In [12]:
import numpy as np
np.random.seed(42)
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import backend as K
import re
import warnings
warnings.filterwarnings('ignore')
import gensim
from gensim.scripts.glove2word2vec import glove2word2vec

import os
os.environ['OMP_NUM_THREADS'] = '4'

In [2]:
train = pd.read_csv('../pre_data/df_train_tw_corr.csv', sep=';', index_col='index')
test = pd.read_csv('../pre_data/df_test_tw_corr.csv', sep=';', index_col='index')
submission = pd.read_csv('../data/sample_submission.csv')

y = pd.read_csv('../data/train.csv')

X_train = train["comment_text"].fillna("fillna").values
y_train = y[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test["comment_text"].fillna("fillna").values


max_features = 200000
maxlen = 460


tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train) + list(X_test))
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [17]:
emb_files = [
    #('wiki.en.vec',False),
 #'deps.words',
 #('glove.twitter.27B.50d.txt',False),
 #('glove.twitter.27B.100d.txt',False),
 #('glove.840B.300d.txt',False),
 #('wiki.de.vec',False),
 #('wiki.fr.vec',False),
 #('glove.twitter.27B.200d.txt',False),
 ('GoogleNews-vectors-negative300.bin',True),
 ('glove.twitter.27B.25d.txt',False)]

In [18]:
for efile, flag_fin in emb_files:
    print(efile)
    try:
        model = gensim.models.KeyedVectors.load_word2vec_format(efile, binary=flag_fin)
    except:
        glove2word2vec(efile, efile[:-4]+'_w2v_'+efile[-4:])
        model = gensim.models.KeyedVectors.load_word2vec_format(efile[:-4]+'_w2v_'+efile[-4:], binary=flag_fin)
    embed_size = model.vector_size
    #def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = {key:model.wv[key] for key in model.wv.vocab.keys()}

    all_embs = np.stack(embeddings_index.values())
    emb_mean, emb_std = all_embs.mean(), all_embs.std()

    word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    np.save('emb_lemm_corr_'+efile[:8]+'_'+str(embed_size)+'_'+'.npy', embedding_matrix)

GoogleNews-vectors-negative300.bin
glove.twitter.27B.25d.txt
